<a href="https://colab.research.google.com/github/ram1014/colab_files/blob/main/Practica6_263165.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Practica 6 - Manejo de Libreria Pandas

##Goole Drive load and pandas import


In [137]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as p
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv('/content/drive/MyDrive/ClassFiles/titanic.csv')



#Ejercicio 1: Análisis de la distribución de supervivencia por combinación de sexo y clase del pasajero.

    

* Calcula la proporción de supervivencia para cada combinación de 'Sex' y 'Pclass'.
* Identifica qué combinación tuvo la tasa de supervivencia más alta.
* Identifica qué combinación tuvo la tasa de supervivencia más baja.


     
    



In [128]:
#survival_count = df.groupby(['Sex','Pclass'])['Survived'].count()
#survival_sum = df.groupby(['Sex','Pclass'])['Survived'].sum()
survival_ratio = df.groupby(['Sex','Pclass']).agg(
        survived_ratio=('Survived', 'mean')
        #total=('Survived', 'count'),
        #survived=('Survived', 'sum'),
        #survived_ratio= ('Survived', lambda x: x.sum()/x.count())
)

id_max = survival_ratio['survived_ratio'].idxmax()
id_min = survival_ratio['survived_ratio'].idxmin()

resultmax = survival_ratio.loc[id_max]
resultmin = survival_ratio.loc[id_min]
print(f"With survival rate of {resultmax['survived_ratio']:.3f} the Sex/Pclass combination with best survival rate is {resultmax.name[0]}/Class {resultmax.name[1]}")
print(f"With survival rate of {resultmin['survived_ratio']:.3f} the Sex/Pclass combination with worst survival rate is {resultmin.name[0]}/Class {resultmin.name[1]}")




With survival rate of 0.968 the Sex/Pclass combination with best survival rate is female/Class 1
With survival rate of 0.135 the Sex/Pclass combination with worst survival rate is male/Class 3


#Ejercicio 2: Identificación de familias grandes a bordo.

    Crea una nueva columna 'FamilySize' sumando las columnas 'SibSp' y 'Parch'.
    Considera como "familia grande" a aquellas donde 'FamilySize' es mayor a 3.
    Calcula el número de pasajeros en familias grandes.
    Calcula la proporción de supervivencia entre los pasajeros que pertenecen a familias grandes.


In [168]:
#create new column FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch']

In [130]:
bigFam = df[df['FamilySize'] > 3]

print(f"Number of passengers with big families is {bigFam.shape[0]}")
survival_ratio_big_fam = df[df['FamilySize']>3].agg(
        survived_ratio= ('Survived', 'mean')
)

print(f"Survival rate for members of big families is:\n {survival_ratio_big_fam['Survived']['survived_ratio']:.3f}")

Number of passengers with big families is 62
Survival rate for members of big families is:
 0.161


#Ejercicio 3: Segmentación por grupos de edad.

Clasifica a los pasajeros en las siguientes categorías de edad(tip puede resultar mas sencillo realizarlo con una función):  menor de edad (< 18) y mayor de edad (>=18)

In [136]:
pass_adult = df[df['Age'] >= 18]
pass_minor = df[df['Age'] < 18]

print(f"Number of adults: {pass_adult.shape[0]}")
print(f"Number of minors: {pass_minor.shape[0]}")

Number of adults: 601
Number of minors: 113


#Ejercicio 4: Comparación entre promedios calculados manualmente y con Pandas

    Utiliza NumPy para calcular el promedio de las columnas 'Age' y 'Fare', ignorando valores nulos.
    Compara estos valores con los promedios obtenidos utilizando los métodos nativos de Pandas.
    Verifica que los resultados sean consistentes.


In [156]:
arr_age = np.array(df['Age'])
arr_filled = np.nan_to_num(arr_age, nan=0.0)
print("NumPy data:")
print(f"Age mean: {arr_filled.mean():.4f}")
arr_fare = np.array(df['Fare'])
arr_fare[np.isnan(arr_fare)] = 0
print(f"Fare mean: {arr_fare.mean():.4f}")


print("Pandas data:")
age_fare_data = df.agg(
        Age_Mean= ('Age', 'mean'),
        Fare_Mean= ('Fare', 'mean')
)

print(age_fare_data)

NumPy data:
Age mean: 23.7993
Fare mean: 32.2042
Pandas data:
                 Age       Fare
Age_Mean   29.699118        NaN
Fare_Mean        NaN  32.204208


#Ejercicio 5. Creación de intervalos de clase usando NumPy y análisis con Pandas

    Divide la columna 'Fare' en 5 intervalos equidistantes utilizando la función numpy.linspace, el estudiante deberá investigar la operación de esta función en python.
    Crea una nueva columna en el DataFrame que asigne a cada pasajero el intervalo correspondiente de su tarifa.
    Calcula el número de pasajeros en cada intervalo utilizando Pandas y la proporción de supervivientes por intervalo.


In [174]:
fare_range = np.linspace(df['Fare'].min(), df['Fare'].max(), 6)
labels = [f'Fare {i+1}' for i in range(6 - 1)]
df['FareRange'] = pd.cut(df['Fare'], bins=fare_range, labels=labels, include_lowest=True)

fare_range_surv_rate = df.groupby('FareRange').agg(
        Surv_rate=('Survived', 'mean')
)

print(fare_range_surv_rate)

           Surv_rate
FareRange           
Fare 1      0.361575
Fare 2      0.757576
Fare 3      0.647059
Fare 4           NaN
Fare 5      1.000000


/tmp/ipython-input-2526045091.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fare_range_surv_rate = df.groupby('FareRange').agg(
